In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
path_new_dem = "/data/ATM/data_1/sfm/agi_projects/test_gcps8/output/test_gcps8_dem_absolute.tif"

In [3]:
import numpy as np
def calculate_bounds(transform_matrix, width, height):
    
    transform_matrix = np.asarray(transform_matrix).reshape(3,3)
        
    # Define the four corners of the image in Cartesian coordinates
    corners = np.array([
        [0, 0],  # Top-left corner
        [width, 0],  # Top-right corner
        [width, height],  # Bottom-right corner
        [0, height]  # Bottom-left corner
    ])

    # Convert corners to homogeneous coordinates by adding a row of ones
    corners_homogeneous = np.vstack((corners.T, np.ones((1, corners.shape[0]))))

    # Apply the transformation matrix to the corners
    transformed_corners_homogeneous = transform_matrix @ corners_homogeneous

    # Convert back to Cartesian coordinates by dividing by the last row
    transformed_corners_cartesian = transformed_corners_homogeneous[:2, :] / transformed_corners_homogeneous[2, :]

    # Calculate the min/max bounds for x and y
    min_x, max_x = np.min(transformed_corners_cartesian[0, :]), np.max(transformed_corners_cartesian[0, :])
    min_y, max_y = np.min(transformed_corners_cartesian[1, :]), np.max(transformed_corners_cartesian[1, :])

    return min_x, min_y, max_x, max_y


In [4]:
import src.load.load_image as li

new_dem, new_transform = li.load_image(path_new_dem, return_transform=True)


new_dem[new_dem == -9999] = np.nan

print(np.nanmin(new_dem), np.nanmax(new_dem))

# get bounds
new_bounds = calculate_bounds(new_transform, new_dem.shape[1], new_dem.shape[0])
print(new_bounds)

132.68611 1320.6384
(-2002172.5958528647, 724012.9429729865, -1982520.1492790317, 729795.1460498965)


In [18]:
import src.dem.estimate_dem_quality_old as edq

qd = edq.estimate_dem_quality(new_dem, modern_source="REMA10", historic_bounds=new_bounds, use_rock_mask=True)

In [19]:
for key, value in qd.items():
    print(key, value)

all_mean_difference -0.28371793
all_std_difference 193.1377
all_mean_difference_abs 157.33104
all_difference_abs_std 112.02312
all_rmse 193.13788
all_mae 157.33104
all_mad 140.42903
all_correlation 0.8318514795651736
rock_mean_difference 11.861684
rock_std_difference 165.83163
rock_mean_difference_abs 133.12477
rock_difference_abs_std 99.59234
rock_rmse 166.25536
rock_mae 133.12477
rock_mad 125.91882
rock_correlation 0.7320582814170166
